In [20]:
from os import getenv
from datetime import datetime, timezone
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_community.chat_models.cloudflare_workersai import ChatCloudflareWorkersAI

load_dotenv()

True

In [21]:
llm = ChatCloudflareWorkersAI(
    account_id=getenv("CLOUDFLARE_ACCOUNT_ID"),
    api_token=getenv("CLOUDFLARE_API_TOKEN"),
    model="@hf/nousresearch/hermes-2-pro-mistral-7b",
)

In [22]:
@tool
def date_time() -> str:
    """
    Function Name: date_time
    Parameters: None
    Description: Returns the current UTC date and time in ISO format.
    """
    return datetime.now(timezone.utc).isoformat()

tools = {
    "date_time": date_time
}

llm = llm.bind_tools(tools.values())

In [23]:


query = "Using the date_time tool, what is the current date?"

messages = [HumanMessage(query)]

ai_msg = llm.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = tools[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

llm.invoke(messages)

2025-03-23 21:42:35 - INFO - Sending prompt to Cloudflare Workers AI: {'prompt': 'role: user, content: Using the date_time tool, what is the current date?', 'tools': [{'type': 'function', 'function': {'name': 'date_time', 'description': 'Function Name: date_time\nParameters: None\nDescription: Returns the current UTC date and time in ISO format.', 'parameters': {'properties': {}, 'type': 'object'}}}]}
2025-03-23 21:42:37 - INFO - Sending prompt to Cloudflare Workers AI: {'prompt': "role: user, content: Using the date_time tool, what is the current date?\nrole: assistant, content: {'result': {'response': None, 'tool_calls': [{'arguments': {}, 'name': 'date_time'}], 'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}}, 'success': True, 'errors': [], 'messages': []}, tools: [{'name': 'date_time', 'arguments': {}}]\nrole: tool, content: 2025-03-24T01:42:37.431549+00:00, tool_call_id: eb031e2e-2c85-49b9-b659-3b01c6a973ea", 'tools': [{'type': 'function', 'function': {'na

[{'name': 'date_time', 'args': {}, 'id': 'eb031e2e-2c85-49b9-b659-3b01c6a973ea', 'type': 'tool_call'}]


AIMessage(content="{'result': {'response': 'The current date, using the date_time tool, is March 24, 2025.', 'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}}, 'success': True, 'errors': [], 'messages': []}", additional_kwargs={}, response_metadata={}, id='run-f0c8df42-50ef-498b-8327-29d631d7c5c6-0')